# 2.6 모델 선택과 훈련

## 2.6.0 이전 내용 복습

### 2.6.0.1 데이터 추출 함수

In [1]:
import os
import tarfile
import urllib

DOWNLOAD_ROOT = 'https://raw.githubusercontent.com/ageron/handson-ml2/master/'
HOUSING_PATH = '../' + os.path.join('datasets', 'housing')
HOUSING_URL = DOWNLOAD_ROOT + 'datasets/housing/housing.tgz'

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    """
    현재 작업 공간에 "datasets/housing" 디렉터리 생성
    housing.tgz 파일 내려받음
    같인 디렉터리에 압축을 풀어 "housing.csv" 파일 생성
    """
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
#fetch_housing_data(HOUSING_URL, HOUSING_PATH)

<br>

### 2.6.0.2 데이터 읽기 함수

In [2]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


<br>

### 2.6.0.3 테스트 세트 분리

In [3]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

# income_cat 특성 생성
housing['income_cat'] = pd.cut(housing['median_income'],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    
# income_cat 특성 삭제
for set_ in (strat_train_set, strat_test_set):
    set_.drop('income_cat', axis=1, inplace=True)
    
print('strat_train_set size : {}'.format(len(strat_train_set)))
print('strat_test_set size : {}'.format(len(strat_test_set)))

strat_train_set size : 16512
strat_test_set size : 4128


<br>

### 2.6.0.4 훈련 세트 예측 변수와 레이블 분리

In [4]:
# drop()은 데이터 복사본을 만들며 strat_train_set에 영향을 주지 않음
housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

<br>

### 2.6.0.5 변환 파이프라인

In [5]:
# 특성 조합 변환기 정의
from sklearn.base import BaseEstimator, TransformerMixin

col_names = "total_rooms", "total_bedrooms", "population", "households"
rooms_ix, bedrooms_ix, population_ix, households_ix = [housing.columns.get_loc(c) for c in col_names]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # *args 나 **kargs 가 아님
        self.add_bedrooms_per_room = add_bedrooms_per_room  
    def fit(self, X, y=None):
        return self # 더 이상 할 일이 없음
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix] # 가구당 방 갯수
        population_per_household = X[:, population_ix] / X[:, households_ix] # 가구당 인구수
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix] # 방 당 화장실 갯수
            return np.c_[X, 
                         rooms_per_household,
                         population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, 
                         rooms_per_household,
                         population_per_household]

In [6]:
# 수치형 특성 변환 파이프라인
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')), # 결측치 처리
    ('attribs_adder', CombinedAttributesAdder()), # 특성 조합
    ('std_scaler', StandardScaler()) # 수치형 특성 데이터 스케일링
])

In [7]:
# 통합 변환 파이프라인
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

housing_num = housing.drop('ocean_proximity', axis=1)
num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs) # 범주형 특성 데이터 숫자 변환
])

housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

array([[-1.15604281,  0.77194962,  0.74333089, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18684903, -1.34218285,  0.18664186, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.        ,
         0.        ,  0.        ],
       [-1.43579109,  0.99645926,  1.85670895, ...,  0.        ,
         1.        ,  0.        ]])

<br>

## 2.6.1 훈련 세트에서 훈련하고 평가하기

### 2.6.1.1 선형 회귀 모델 훈련 및 평가

In [8]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

<br>

- 훈련 세트에 있는 몇 개 샘플에 대해 적용

In [9]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print('예측 : ', lin_reg.predict(some_data_prepared))
print('레이블 : ', list(some_labels))

예측 :  [210644.60459286 317768.80697211 210956.43331178  59218.98886849
 189747.55849879]
레이블 :  [286600.0, 340600.0, 196900.0, 46300.0, 254500.0]


<br>

- 사이킷런의 `mean_square_error` 함수를 사용해 전체 훈련 세트에 대한 이 회귀 모델의 RMSE 측정

In [10]:
from sklearn.metrics import mean_squared_error


housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

68628.19819848923

<br>

- 없는 것 보다는 낫지만 확실히 좋은 점수는 아니다.
- 대부분 구역의 중간 주책 가격은 <span>&#36;</span>120,000 에서 <span>&#36;</span>265,000 사이 이다.
- 그러므로 예측 오차가 <span>&#36;</span>68,628 인 것은 매우 만족스럽지 못하다.

- 이는 모델이 훈련 데이터에 **과소적합**된 사례이다.
- 이런 상황은 다음과 같은 사실들을 말해준다.
  - 특성들이 좋은 예측을 만들 만큼 충분한 정보를 제공하지 못함
  - 모델이 충분히 강력하지 못함  
  
  
- 과소 적합을 해결하는 주요 방법
  - 더 강력한 모델 선택
  - 훈련 알고리즘에 더 좋은 특성 주입
  - 모델의 규제 감소  
  
  
- 이 모델은 규제를 사용하지 않았으므로 마지막 옵션은 제외된다.
- 특성을 더 많이 추가할 수 있다.(ex. 로그 스케일된 인구)
- 먼저 더 복잡한 모델을 시도해보자.

<br>

### 2.6.1.2 의사결정나무 회귀 모델 훈련 및 평가

- `DecisionTreeRegressor` 를 훈련해보자.
- 이 모델은 강력하고 데이터에서 복잡한 비선형 관계를 찾을 수 있다.

In [11]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

In [12]:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

<br>

- 모델이 데이터에 너무 심하게 **과대적합**된 것으로 보임

- 확신이 드는 모델이 론칭할 준비가 되기 전까지 테스트 세트는 사용하지 않으려 한다.
- 그러므로 훈련 세트의 일부분으로 훈련을 하고, 다른 일부분은 모델 검증에 사용해야 한다.

<br>

## 2.6.2 교차 검증을 사용한 평가

### 2.6.2.1 간단한 방법

- 결정 트리 모델을 평가하는 방법을 생각해보자.
- 우선 `train_test_split` 함수를 사용해 훈련 세트를 더 작은 훈련 세트와 검증 세트로 나눈다.
- 더 작은 훈련 세트에서 모델을 훈련시킨다.
- 검증 세트로 모델을 평가한다.  
  
  
- 조금 수고스럽지만 어렵지 않으며 매우 잘 동작한다.

<br>

### 2.6.2.2 K-겹 교차 검증(k-fold cross-validation)

- 훌륭한 대안으로 사이킷런의 **K-겹 교차 검증(k-fold cross-validation)** 기능을 사용하는 방법도 있다.

**K-겹 교차 검증(k-fold cross-validation) 수행 과정 예시**

- 훈련 세트를 **폴드(fold)**라 불리는 10개의 서브셋으로 무작위로 분할한다.
- 그런 다음 결정 트리 모델을 10번 훈련하고 평가한다.
- 매번 다른 폴드를 선택해 평가에 사용하고, 나머지 9개 폴드는 훈련에 사용한다.
- 10개의 평가 점수가 담긴 배열이 결과가 된다.

<br>

### 2.6.2.3 K-겹 교차 검증을 이용한 의사결정나무 회귀 모델 평가

In [17]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg,
                         housing_prepared,
                         housing_labels,
                         scoring='neg_mean_squared_error',
                         cv=10)
tree_rmse_scores = np.sqrt(-scores)
tree_rmse_scores

array([70214.92154654, 66870.18406577, 71246.32830842, 68564.8764963 ,
       72161.02228554, 75405.4042396 , 71266.09364549, 70108.57086478,
       75151.36627889, 70307.09156501])

<br>

- 사이킷런의 교차 검증 기능은 `scoring` 매개변수에 (낮을수록 좋은) 비용 함수가 아니라 (클수록 좋은) 효용 함수를 기대한다.
- 그래서 평균 제곱 오차(MSE)의 반댓값(즉, 음숫값)을 계산하는 `neg_mean_squared_error` 함수를 사용한다.
- 이런 이유로 `tree_rmse_scores`를 계산할 때 제곱근을 계산하기 전에 `-scores`로 부호를 바꿨다.
  - 평균 제곱 오차(MSE)가 작을수록 좋은 비용 함수이다.
  - 그러므로 부호가 반대가 되어야 `scoring` 매개변수 정의에 맞다.
  - 회귀 모델에서 `scoring` 매개변수를 지정하지 않으면 기본적으로 0~1 사이의 값을 가지는 `r2_score`가 사용된다.

<br>

- 결과 확인

In [18]:
def display_scores(scores):
    print('점수:', scores)
    print('평균:', scores.mean())
    print('표준편차:', scores.std())
    
display_scores(tree_rmse_scores)

점수: [70214.92154654 66870.18406577 71246.32830842 68564.8764963
 72161.02228554 75405.4042396  71266.09364549 70108.57086478
 75151.36627889 70307.09156501]
평균: 71129.58592963338
표준편차: 2506.5769863471382


<br>

- 결정 트리 결과가 이전만큼 좋아 보이지 않는다. (선형 회귀 모델보다 나쁘다.)
- 교차 검증으로 모델의 성능을 추정하는 것 뿐만 아니라, 이 추정이 얼마나 정확한 지(를 표준 편차를 통해) 측정할 수 있다.  
  
  
- K-겹 교차 검증은 모델을 여러 번 훈련 시켜야 해서 비용이 비싸므로 언제나 쓸 수 있는 것은 아니다.

<br>

### 2.6.2.4 K-겹 교차 검증을 이용한 선형 회귀 모델 평가

In [19]:
lin_scores = cross_val_score(lin_reg,
                             housing_prepared,
                             housing_labels,
                             scoring='neg_mean_squared_error',
                             cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

점수: [66782.73843989 66960.118071   70347.95244419 74739.57052552
 68031.13388938 71193.84183426 64969.63056405 68281.61137997
 71552.91566558 67665.10082067]
평균: 69052.46136345083
표준편차: 2731.6740017983484


<br>

- 결정 트리 모델이 과대적합 되어 선형 회귀 모델보다 성능이 나쁘다.

<br>

### 2.6.2.5 K-겹 교차 검증을 이용한 랜덤포레스트 회귀 모델 평가

- `RandomForestRegressor` 모델의 평가를 해보자.
- 랜덤 포레스트는 특성을 무작위로 선택해서 많은 결정 트리를 만들고 그 예측을 평균 내는 방식으로 작동한다.
- 여러 다른 모델을 모아서 하나의 모델을 만드는 것을 **앙상블 학습** 이라고 한다.
- 앙상블 학습은 머신러닝 알고리즘의 성능을 극대화하는 방법 중 하나이다.

In [20]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
forest_scores = cross_val_score(forest_reg,
                                housing_prepared,
                                housing_labels,
                                scoring='neg_mean_squared_error',
                                cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

C:\Users\shkim\AppData\Local\conda\conda\envs\python36\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


점수: [52776.85217055 50081.60851963 52941.37089774 55695.27190682
 52343.00592468 55472.82770565 52459.67437438 51604.92531235
 56385.93871695 53100.12300553]
평균: 53286.15985342818
표준편차: 1878.168417541856


<br>

- 이전 모델들 보다 훌륭한 성능을 나타낸다.
- 하지만 훈련 세트에 대한 점수가 검증 세트에 대한 점수보다 훨씬 낮으므로 이 모델도 여전히 훈련 세트에 과대 적합되어 있다.

**과대 적합 해결 방법**

- 모델을 간단히 하기
- 제한을 적용 (즉, 규제)
- 더 많은 훈련 데이터를 모음

- 여러 종류의 머신러닝 알고리즘으로 하이퍼파라미터 조정에 너무 많은 시간을 들이지 않으면서 다양한 모델(ex. 다양한 커널의 서포트 벡터 머신, 신경망 등)을 시도해봐야 한다.
- 가능성 있는 2~5개 정도의 모델을 선정하는 것이 목적이다.

<br>

### 2.6.2.6 학습한 모델 저장

- 실험한 모델을 모두 저장해두면 필요할 때 쉽게 모델을 복원할 수 있다.
- 저장해야 할 것들
  - 교차 검증 점수
  - 실제 예측값
  - 하이퍼파라미터
  - 훈련된 모델 파라미터
- 이렇게 하면 여러 모델의 점수와 모델이 만든 오차를 쉽게 비교할 수 있다.
- 파이썬의 `pickle` 패키지나 `joblib` 를 사용하여 사이킷런 모델을 쉽게 저장할 수 있다.
  - `joblib` : 큰 넘파이 배열을 저장하는 데 아주 효율적인 라이브러리 (`pip` 를 통해 설치 가능)
  
```python
import joblib

joblib.dump(my_model, "my_model.pkl")

# 그리고 나중에...
my_model_loaded = joblib.load("my_model.pkl")
```